# Lab 4. Classification  and regression using KNN and SVM



In [0]:
# Some IPython magic
# Put these at the top of every notebook, here nbagg is used for interactive plots
%reload_ext autoreload
%autoreload 2
%matplotlib nbagg

import numpy as np
import matplotlib.pyplot as plt

## KNN Classification
KNN is a non-parametric, instance-based, supervised learning algorithm.

It doesn't learn a function, but memorizes the training set, and uses it at inference time.

Pseudocode:

- Train time: store all training points.
- Test time: find the closest k points , and output the most similar class.
### Implement your own knn

Let's say you have the following dataset:

In [13]:
# load data
# pickle is a python data file used to store objects on disk.
import pickle
data = pickle.load(open('data-knn.pkl', 'rb'))
X = data['data']
y = data['target']

FileNotFoundError: ignored

In [0]:
# Plot the dataset. The dataset is 2D.
plt.scatter(X[:,0], X[:,1], c=y)
plt.show()

Implement knn. Implement a function **`pairwise_distance_matrix(X,Y)`** in numpy, that computes the distance between any point in X with any point in Y. Try implementing this function with no for loops.

Hint. You need to use numpy's broadcasting.

In [0]:
# Implement your own euclidean distance method using numpy

def pairwise_distance_matrix(X, Y):
    """Compute the pairwise distance between rows of X and rows of Y
    Arguments
    ----------
    X: ndarray of size (N, D)
    Y: ndarray of size (M, D)
    Returns
    --------
    distances: matrix of shape (N, M), each entry D[i,j] is the distance between
    X[i,:] and Y[j,:] using the dot product.
    """
    #########################
    # Compute distance_matrix
    #########################
    
#     distance_matrix = np.sqrt(np.square(X).dot(np.square(Y).T))
    distance_matrix = np.sqrt(np.square(X - Y))
    print(distance_matrix.shape)
    
    return distance_matrix


Now implement KNN such that it takes as input the training dataset. Try to use numpy functions as much as possible.

In [0]:
# Implement your own version of KNN.

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels

class myKNN(BaseEstimator, ClassifierMixin):
    def __init__(self, n_neighbors=3):
        self.n_neighbors = n_neighbors
        
    def fit(self, X, y):
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        # Store the classes seen during fit
        self.classes_ = unique_labels(y)
        self.X_ = X
        self.y_ = y
        # Return the classifier
        return self
    
    def predict(self, X):
        # Check is fit had been called
        check_is_fitted(self, ['X_', 'y_'])
        # Input validation
        X = check_array(X)
        # Implement knn predict
        
#         X  = np.array([[1, 2], [3, 4]])
#         self.X_ = np.array([[1, 2], [3, 4]])
        
        distance_matrix = pairwise_distance_matrix(X, self.X_) # compute distance
        print(distance_matrix)
        idx = distance_matrix.argsort() # sort and return indices
        print(idx)
        classes_pred = y[idx] # return class of that index
        classes_n = classes_pred[:, :self.n_neighbors] # top n_neighbors
        classes_count = [np.bincount(classes_n[i]) for i in range(classes_n.shape[0])]
        classes = np.array([np.argmax(classes_count[i]) for i in range(len(classes_count))])
        
        return classes
    
    def predict_proba(self, X):
        return self.predict(X)


Test the accuracy of your implementation.

In [0]:
# Find the accuracy of your KNN implementation
from sklearn.metrics import accuracy_score


Now train the sklearn KNN

In [0]:
# Test sklearn's KNN implementation
# Hint! Use algorithm = 'brute' for first try
# Try to improve the score using other parameters for 'metric', 'algorithm' and 'n_neighbors'. 
from sklearn.neighbors import KNeighborsClassifier


#### Finding the best k
k is a hyperparameter for knn.

A hyperparameter is a parameter that is not learned from the data. In order to find the best hyperparameters you need to train, and measure the validation accuracy for multiple values of the hyperparameters.

In [0]:
# Implement a method that will find the best K parameter for a classifier.
# Plot every pair of score and K.
# Find best K parameter for the classifier declared before.
def find_best_k(clf, max_k=30):
    scores = []
    ks = np.arange(1,max_k+1)
    #############
    # train knn for diferent values of k and store the validation accuracy

    plt.xticks(range(1,max_k+1))
    plt.plot(range(1,max_k+1), scores)
    
    max_score = max(scores)
    
    return scores.index(max_score) + 1, max_score



Compare the two implementations of knn on the wine dataset. Don't forget to split the dataset into train and test sets.

In [67]:
# Load wine dataset and partition it in train and test splits.
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

wine = load_wine()
X = wine.data
y = wine.target

X_train, X_test, y_train, y_test = train_test_split(X, y)

my_clf = myKNN()
my_clf.fit(X_train, y_train)
my_y_pred = my_clf.predict(X_test)
print(my_y_pred)

clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(y_pred)

ValueError: ignored

In [0]:
# Test the accuracy of your KNN implementation and sklearn's KNN on wine dataset.


Note that KNN relies on a distance measure. Therefore you need to normalize the data. Try the algorithms again after normalizing the data.

In [0]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler


In [0]:
## SVM Classification

SVM is a classification algorithm that tries to find a separating hyperplane between two classes. It is therefore a linear algorithm. In order to obtain a non-linear separating plane, we can transform the feature space. This can be done using the kernel trick (see lecture slides).

In [0]:
# some datasets that we will use.
from sklearn import datasets
circles = datasets.make_circles(n_samples=200, factor=.5,
                                      noise=.05)
moons = datasets.make_moons(n_samples=200, noise=.05)
blobs = datasets.make_blobs(n_samples=200, random_state=9, centers=2, cluster_std=2)

In [0]:
datasets  = [circles, moons, blobs]
fig, axes = plt.subplots(3,1, figsize=(5,12))
for i, (X, y) in enumerate(datasets):
    axes[i].scatter(X[:,0],X[:,1],c=y)
plt.show()

Apply SVM to separate the blobs dataset and plot the decision boundary.

In [0]:
from sklearn.svm import SVC
clf = SVC(kernel='linear')


Now apply SVM on moons and circles datasets. Try different kernels. Which one works best?

In [0]:
# apply svm for the datasets above

### Hyperparameter tuning
What if we had non-separable data? We need to find the best kernel with the best hyperparameters. We need to set up an experiment, split the dataset into train and test set, measure accuracy on test set to evaluate performance, and tune the hyperparameters to find the model that works best. 

In [0]:
from sklearn import datasets
circles = datasets.make_circles(n_samples=200, factor=.5,
                                      noise=.25)
moons = datasets.make_moons(n_samples=200, noise=.25)
blobs = datasets.make_blobs(n_samples=200, random_state=3, centers=2, cluster_std=2.2)
datasets  = [circles, moons, blobs]
fig, axes = plt.subplots(3,1, figsize=(5,12))
for i, (X, y) in enumerate(datasets):
    axes[i].scatter(X[:,0],X[:,1],c=y)
plt.show()

#### Grid search
In order to tune a hyperparameter you should set up a validation experiment, and loop over different values of that hyperparameter, calling the experiment to train, test, score the model. If there are more hyperparameters, you could use nested loops. However, sklearn provides a tool to tune those parameters called [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Use grid search to find the best parameters for the 3 datasets (circles, moons and blobs).

## Regression
Both KNN and SVM can be adapted to be applied on regression problems. Both algorithms rely on distance metrics, therefore make sure you normalize your dataset. You can research different normalization techniques [here](http://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#sphx-glr-auto-examples-preprocessing-plot-all-scaling-py).


To Do: a regression problem to on which to apply KNN and SVM regression

In [0]:
# Apply KNN regression and SVM regression on the following dataset
dataset = fetch_california_housing()
X, y = dataset.data, dataset.target
